## [Data Science] Image Processing : Exploratory & Modeling
In this poject we will work on a set of image of letters in plenty of differents fronts.
The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labeled examples. Given these sizes, it should be possible to train models quickly on any machine.


You need to go step by step through this notebook, and fill the code in the //TODO. 
Please make sure to explain clearly all your choices. 

In [ ]:

# These are all the modules that could be use for this project. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import imageio
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os
import sys
import tarfile
from IPython.display import display, Image
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

%matplotlib inline
np.random.seed(133)
print(f'Using Python  = {sys.version.split()[0]}')
print(f'Tensorflow    = {tf.__version__}')


## Step 1 Download data set

In [ ]:
In order to save you time we provide this function

In [ ]:

url = 'https://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None
data_root = '.' # Change me to store data elsewhere

def download_progress(count, blockSize, totalSize):
  """
  Simple hook to report the progress of a download. 
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
        
def gz_download(filename, expected_bytes, force=False):
  """
  Download a file if not present, and make sure it's the right size.
  """
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress)
    print('\nDownload Complete!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', dest_filename)
  else:
    raise Exception(
      'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

train_filename = gz_download('notMNIST_large.tar.gz', 247336696)
test_filename = gz_download('notMNIST_small.tar.gz', 8458043)

Extract the dataset from the compressed .tar.gz file. This should give you a set of directories, labeled A through J.

In [ ]:
def tar_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  
  // TODO : add your code here

  print(data_folders)
  return data_folders
  
train_folders = tar_extract(train_filename)
test_folders = tar_extract(test_filename)

## Problem 1
Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. 

In [ ]:
// TODO: Here your code and explaination


Now let's load the data in a more manageable format. Since, depending on your computer setup you might not be able to fit it all in memory, we'll load each class into a separate dataset, store them on disk and curate them independently. We could merge into a single dataset of manageable size if needed.

We'll convert the entire dataset into a 3D array (image index, x, y) of floating point values, normalized to have approximately zero mean and standard deviation ~0.5 to make training easier down the road.


In [ ]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load_letters(folder, min_num_images):
  """Load the data for a single letter label."""
  image_files = os.listdir(folder)
  dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
  print(folder)
  num_images = 0
  for image in image_files:
    image_file = os.path.join(folder, image)
    try:
      image_data = (imageio.imread(image_file).astype(float) - 
                    pixel_depth / 2) / pixel_depth
      if image_data.shape != (image_size, image_size):
        raise Exception('Unexpected image shape: %s' % str(image_data.shape))
      dataset[num_images, :, :] = image_data
      num_images = num_images + 1
    except IOError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    except ValueError as e:
      print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    
    
  dataset = dataset[0:num_images, :, :]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' %
                    (num_images, min_num_images))
    
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  return dataset
        
def pickle(data_folders, min_num_images_per_class, force=False):
  dataset_names = []
  for folder in data_folders:
    set_filename = folder + '.pickle'
    dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
      # You may override by setting force=True.
      print('%s already present - Skipping pickling.' % set_filename)
    else:
      print('Pickling %s.' % set_filename)
      dataset = load_letters(folder, min_num_images_per_class)
      try:
        with open(set_filename, 'wb') as f:
          pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
      except Exception as e:
        print('Unable to save data to', set_filename, ':', e)
  
  return dataset_names

train_datasets = pickle(train_folders, 45000)
test_datasets = pickle(test_folders, 1800)

## Problem 2
Verify that the data still looks good. Displaying a sample of the labels and images from the ndarray.

In [ ]:
// TODO: Here your code and explaination

## Problem 3
Another check: Verify that the data is balanced across classes. 

In [ ]:
// TODO: Here your code and explaination


Merge and prune the training data as needed. Depending on your computer setup, you might not be able to fit it all in memory, and you can tune train_size as needed. The labels will be stored into a separate array of integers 0 through 9.

Also create a validation dataset for hyperparameter tuning.

In [ ]:

# For example
train_size = 200000
valid_size = 10000
test_size = 10000

// TODO: Here your code and explaination

print('Training:', train_dataset.shape, train_labels.shape)
print('Validation:', valid_dataset.shape, valid_labels.shape)
print('Testing:', test_dataset.shape, test_labels.shape)

Next, we'll randomize the data. It's important to have the labels well shuffled for the training and test distributions to match.



In [ ]:
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels
train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)
valid_dataset, valid_labels = randomize(valid_dataset, valid_labels)

## Problem 4
Convince yourself that the data is still good after shuffling!



In [ ]:
// TODO: Here your code and explaination

Save the data for later reuse:



In [ ]:

pickle_file = os.path.join(data_root, 'notMNIST.pickle')

try:

// TODO: Here your code and explaination

except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [ ]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

## Problem 5
By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set. Measure how much overlap there is between training, validation and test samples.

Optional questions:

What about near duplicates between datasets? (images that are almost identical)
Create a sanitized validation and test set, and compare your accuracy on those in subsequent questions.

In [ ]:
// TODO: Here your code and explaination
# Should render something like
#Duplicates inside training set:  ZZZZ
#Duplicates between training and validation:  ZZZZ
#Duplicates between training and test:  ZZZZ

## Problem 6
Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

In [ ]:
from sklearn.linear_model import LogisticRegression

// TODO: Here your code and explaination


print('Training score = ', train_score)
print('Test score = ', test_score)

## Problem 7
Going on step deeper. 
Please propose a model of your choice.
If you run out of time, you do not have to implement the full code, but explain the steps you need to implement and every parameter choice you make.

We first need to reformat the data

Reformat into a shape that's more adapted to the models we're going to train:

 - data as a flat matrix,
 - labels as float 1-hot encodings.

In [ ]:
// TODO: Here your code and explaination

You can choose a simple multinomial logistic regression using simple gradient descent, or any other multi layer version. Just don't  overengineer and make sure that this model improves your validation / test accuracy from previous problems.

In [ ]:

// TODO: Here your code and explaination